In [4]:
# Downloaded all the necessary libararies
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
import torch

# Loading the training and testing data from CSV files
train_df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test_df = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

# Filling missing values in 'keyword' and 'location' columns with empty strings
train_df['keyword'].fillna('', inplace=True)
train_df['location'].fillna('', inplace=True)

# Combining 'keyword', 'location', and 'text' columns into a single text column for processing
train_df['combined_text'] = train_df['keyword'] + " " + train_df['location'] + " " + train_df['text']
test_df['combined_text'] = test_df['keyword'] + " " + test_df['location'] + " " + test_df['text']

# Preprocessing the combined text by converting to lowercase and removing special characters
train_df['combined_text'] = train_df['combined_text'].str.lower().str.replace('[^a-z0-9\s]', '', regex=True)
test_df['combined_text'] = test_df['combined_text'].str.lower().str.replace('[^a-z0-9\s]', '', regex=True)

# Splitting the training data into a training set and a validation set for model evaluation
train_text, val_text, train_labels, val_labels = train_test_split(
    train_df['combined_text'], train_df['target'], test_size=0.1
)

# Initializing the tokenizer from the DistilBERT model, which is a lighter version of BERT
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Tokenizing the training and validation text
# This will convert text to a format that the model can understand
train_encodings = tokenizer(train_text.tolist(), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_text.tolist(), truncation=True, padding=True, max_length=128)

# Defining a custom dataset class to handle the PyTorch data loading
class DisasterDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Creating dataset objects for training and validation
train_dataset = DisasterDataset(train_encodings, train_labels.tolist())
val_dataset = DisasterDataset(val_encodings, val_labels.tolist())

# Initializing the DistilBERT model for sequence classification with two labels
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)



/tmp/ipykernel_34/589905527.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['keyword'].fillna('', inplace=True)
/tmp/ipykernel_34/589905527.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'd

In [5]:
# Defining a function to compute evaluation metrics (F1 score and accuracy)
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1}

# Setting up training arguments, including directory for results, number of epochs, batch size, etc.
training_args = TrainingArguments(
    output_dir='/kaggle/working/results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/kaggle/working/logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    report_to="none"
)

# Initializing the Trainer with the model, training arguments, datasets, and evaluation metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Training the model on the dataset

# Also downloaded transformers 
# pip install torch

In [6]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.395000,0.480632,0.780840,0.765120
2,0.377200,0.430834,0.809711,0.778626
3,0.289700,0.473524,0.825459,0.801196


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=645, training_loss=0.39920383386833724, metrics={'train_runtime': 95.2421, 'train_samples_per_second': 215.797, 'train_steps_per_second': 6.772, 'total_flos': 324372556873404.0, 'train_loss': 0.39920383386833724, 'epoch': 3.0})